In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import time
from flask import Flask, render_template, request
import threading
import os
import random

# Path to the pre-trained model
path_to_model = "C:\\Users\\ADMIN\\projests\\Wild-Animal-Detection-and-Alert-System-using-YOLO-v8-Model-main\\animal_intrussion.h5"

# Email configuration
sender_email = "kausthubakausthubaam8825@gmail.com"
sender_password = "lmtu bxse uxik sgtm"
recipient_email = "4su21ad022@sdmit.in"
smtp_server = "smtp.gmail.com"
smtp_port = 587

# Class labels and wild animals
class_labels = ['HUMAN', 'TIGER', 'LION', 'CHEETCH', 'HYENA', 'DONKEY', 'GOAT', 'DEER', 'HOUSE', 'ELEPHANT', 'FOX', 'LEOPARD', 'RHINOCEROS', 'SNAKE', 'WOLF', 'CAT', 'CHIMPANZEE', 'COW', 'DOG', 'DUCK', 'MONKEY', 'ZEBRA']
wild_animals = ['TIGER', 'LION', 'CHEETCH', 'HYENA', 'ELEPHANT', 'FOX', 'LEOPARD', 'RHINOCEROS', 'SNAKE', 'WOLF']

# Load the pre-trained model
model = load_model(path_to_model)

# Initialize the webcam
cap = cv2.VideoCapture(0)
last_detection_time = time.time()

# Dummy function to get location data
def get_location():
    # Replace this with actual GPS data retrieval code
    # For demonstration, we use random coordinates
    lat = 28.7041 + (random.random() - 0.5) * 0.1
    lon = 77.1025 + (random.random() - 0.5) * 0.1
    return lat, lon

def send_email_alert(class_label, lat, lon, image_path):
    # Set up the email content
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = f"🚨 ALERT: {class_label} Detected 🚨"

    body = f'''
    🚨 ALERT: {class_label} Detected 🚨
    📅 Timestamp: {time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())} 🕒
    Take immediate precautions for your safety:
    ⿡ Stay calm and alert.
    ⿢ Follow official instructions.
    ⿣ Take necessary precautions.
    ⿤ Stay informed from reliable sources.
    ⿥ Help others while maintaining distance.
    Cooperate with authorities and stay safe! 
    We will provide further updates as soon as possible. Stay safe and stay vigilant!
    
    Click the link below to view the location:
    http://127.0.0.1:5000/?lat={lat}&lon={lon}
    '''
    msg.attach(MIMEText(body, 'plain'))

    # Attach the image
    attachment = open(image_path, 'rb')
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', f'attachment; filename= {os.path.basename(image_path)}')
    msg.attach(part)

    # Send the email
    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(sender_email, sender_password)
    server.sendmail(sender_email, recipient_email, msg.as_string())
    server.quit()

def run_flask():
    app = Flask(__name__)

    @app.route('/')
    def index():
        lat = request.args.get('lat', type=float, default=51.505)
        lon = request.args.get('lon', type=float, default=-0.09)
        return render_template('index.html', lat=lat, lon=lon)

    app.run(debug=True, use_reloader=False)

if not os.path.exists('templates'):
    os.makedirs('templates')

# Create a Flask app template file
with open('templates/index.html', 'w') as f:
    f.write('''
    <!DOCTYPE html>
    <html>
    <head>
        <title>Real-time Location Detection</title>
        <link rel="stylesheet" href="https://unpkg.com/leaflet@1.7.1/dist/leaflet.css" />
        <script src="https://unpkg.com/leaflet@1.7.1/dist/leaflet.js"></script>
        <style>
            #map {
                height: 100%;
                width: 100%;
            }
            html, body {
                height: 100%;
                margin: 0;
                padding: 0;
            }
        </style>
    </head>
    <body>
        <h1>Real-time Location Detection</h1>
        <div id="map"></div>

        <script>
            var lat = {{ lat }};
            var lon = {{ lon }};
            var map = L.map('map').setView([lat, lon], 13);

            L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
                attribution: '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'
            }).addTo(map);

            function onLocationFound(e) {
                console.log("Location found:", e);
                var radius = e.accuracy / 2;

                L.marker(e.latlng).addTo(map)
                    .bindPopup("You are within " + radius + " meters from this point").openPopup();

                L.circle(e.latlng, radius).addTo(map);
            }

            function onLocationError(e) {
                console.log("Location error:", e);
                alert(e.message);
            }

            map.on('locationfound', onLocationFound);
            map.on('locationerror', onLocationError);

            map.locate({setView: true, maxZoom: 16});
        </script>
    </body>
    </html>
    ''')

# Initialize Flask in a separate thread
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

# Wait for Flask app to start
time.sleep(5)

# Camera capture and detection loop
while True:
    # Read frame from webcam
    ret, frame = cap.read()
    
    # Preprocess the image
    img = cv2.resize(frame, (224, 224))
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.0
    
    # Make prediction
    prediction = model.predict(img_tensor)
    class_index = np.argmax(prediction[0])
    class_label = class_labels[class_index]
    
    # Display prediction on video feed
    cv2.putText(frame, class_label, (10, 30), cv2.FONT_HERSHEY_COMPLEX, 1.0, (0, 0, 255), 3)
    cv2.imshow('Webcam', frame)
    
    # Check if a wild animal is detected
    if class_label in wild_animals and time.time() - last_detection_time >= 5:
        # Save the frame
        image_path = 'captured_image.jpg'
        cv2.imwrite(image_path, frame)
        
        # Get the location
        lat, lon = get_location()
        
        # Send email alert with the link to the Flask app
        send_email_alert(class_label, lat, lon, image_path)
        
        last_detection_time = time.time()
    
    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


1/1 [==============================] - 0s 94ms/step


127.0.0.1 - - [23/Jul/2024 15:43:37] "GET /?lat=28.67053654969694&lon=77.0872132537268 HTTP/1.1" 200 -


1/1 [==============================] - 0s 41ms/step


...

1/1 [==============================] - 0s 50ms/step


127.0.0.1 - - [23/Jul/2024 15:43:45] "GET /?lat=28.67053654969694&lon=77.0872132537268 HTTP/1.1" 200 -


1/1 [==============================] - ETA: 0s

127.0.0.1 - - [23/Jul/2024 15:43:47] "GET /?lat=28.67053654969694&lon=77.0872132537268 HTTP/1.1" 200 -


1/1 [==============================] - 0s 41ms/step
